# Steps
✅ Get dataset of 20 stock from database (.csv) <br>
⬜ 

# Further ideas
1. Give multiple subsets of stocks (e.g. "Becky" as a strategy)


# Selected stocks
20 stocks were randomly selected from the S&P 500 pool

In [3]:
stock_symbols = ['MCD', 'ESS', 'ABC', 'PFE', 'MA', 'GWW', 'AOS', 'GD', 'YUM', 'LOW', 'ALB', 'IFF', 'TPR', 'CB', 'ANTM', 'WAT', 'CPB', 'IPGP', 'SBUX', 'HSY']

## Imports

In [1]:
from typing import Dict

from collections import namedtuple
import requests
import os
import pandas as pd
import time
from alpha_vantage.timeseries import TimeSeries

## Getting the data

In [ ]:
full_data = []

for symbol in stock_symbols:
    query_params["symbol"] = symbol
    resp = requests.get(url=url, params=query_params)
    resp.raise_for_status()

In [2]:
API_KEY = "1077F37TLBGNSVY2"

STANDARD_DELAY = 60 // 5
DATA_DIR_PATH = r"C:\Users\Jonas\Desktop\CGI\dev\exp\data"
ts = TimeSeries(key=API_KEY, output_format="pandas")

CSV_COLS = ["symbol", "date", "1. open", "2. high", "3. low", "4. close", "5. adjusted close",
           "6. volume", "7. dividend amount", "8. split coefficient"]

### Writing from av API to .csv files

In [9]:
master_df = pd.DataFrame(columns=CSV_COLS)

for idx, symbol in enumerate(stock_symbols):
#     file_path = os.path.join(DATA_DIR_PATH, f"{symbol}.csv")
    df, _ = ts.get_daily_adjusted(symbol=symbol, outputsize="full")
    df["symbol"] = symbol
    master_df.append(other=df)
    print(f"Appended stock {idx}: <{symbol}>. Waiting...")
    time.sleep(STANDARD_DELAY)

Appended stock 0: <MCD>. Waiting...
Appended stock 1: <ESS>. Waiting...


KeyboardInterrupt: 

### Merging all .csv files

In [3]:
file_names = [os.path.join(DATA_DIR_PATH, f) for f in os.listdir(DATA_DIR_PATH) \
                  if os.path.isfile(os.path.join(DATA_DIR_PATH, f))]

In [4]:
df_list = []
for f in file_names:
    df = pd.read_csv(f, index_col=None, header=0)
    df['symbol'] = f.split("\\")[-1].split(".")[0]
    df_list.append(df)
    
df = pd.concat(df_list, axis=0, ignore_index=True)

### Preprocessing

In [5]:
PRICE_COLUMNS = [
    'date',
    'open',
    'high',
    'low',
    'close',
    'adjusted_close',
    'volume',
    'dividend_amount',
    'split_coefficient',
    'symbol',
]

In [6]:
cols_to_rename = {}
for c in df.columns:
    cols_to_rename[c] = "".join(list(filter(lambda s: s.isalpha() or s == " ", c))).strip()
    cols_to_rename[c] = cols_to_rename[c].replace(" ", "_")
    
df.rename(columns=cols_to_rename, inplace=True)

In [7]:
df['date'] = pd.to_datetime(df['date'])
df['symbol'] = df['symbol'].astype("string") # even though this doesn't do anything
df.dtypes

date                 datetime64[ns]
open                        float64
high                        float64
low                         float64
close                       float64
adjusted_close              float64
volume                      float64
dividend_amount             float64
split_coefficient           float64
symbol                       string
dtype: object

# Algorithm

## Class definitions

In [29]:
class Market:
    def __init__(self, prices: pd.DataFrame, name: str = None):
        assert not prices.empty and all([c in PRICE_COLUMNS for c in prices.columns]), \
            "Wrong DataFrame format!"
        self.prices = prices
        # sort by date ascending, then by symbol descending
        self.prices.sort_values(by=["date", "symbol"], inplace=True, ascending=[True, False])
        self.name = name or "default_market"
        self.max_date = None
    
    @property
    def available_symbols(self):
        return list(self.prices.symbol.unique())
    
    @property
    def current_date(self):
        return self.max_date or "Market has not yet started!"
    
    def pay_dividends(self, stock_portfolio):
        dividend_amount = 0
        for stock, amount in stock_portfolio.items():
            dividend_mask = (self.prices.date == self.max_date) & (self.prices.symbol == stock)
            dividend_amount += self.prices.loc[dividend_mask].dividend_amount.sum() * amount
        if dividend_amount:
            print(f"Paid agents dividends of {dividend_amount}!")
        return dividend_amount
    
    def __iter__(self):
        self.curr = 1
        return self
    
    def __len__(self):
        return len(self.prices)
    
    def __next__(self) -> pd.DataFrame:
        if self.curr < len(self.prices.date.unique()):
            
            self.max_date = self.prices.date.sort_values().unique()[self.curr - 1]
            state = self.prices.loc[self.prices.date <= self.max_date]
            self.curr += 1
            return state
        else:
            #  we are on the last date of the prices > exit the iteration
            raise StopIteration
    
    def __str__(self):
        return f"Market [{self.name}] (current market_date: {self.max_date})"

#### Calculation of dividend yield
__On certain date (choose ~~01/02~~ first trading date of year):__
1. Get the last paid dividend amount from prev. year := `dividend_payment`
2. Multiply by the number of times dividends were paid last year (mostly: 4) := `dividend_total`
3. divide `dividend_total` by `current_share_price`

### Strategy Interface

In [10]:
class Strategy:
    def __init__(self, name: str = None):
        self.name = name or type(self).__name__
    
    def weight(self, market_state: pd.DataFrame) -> Dict[str, float]:
        pass

__Desired code behavior:__
1. ✅ Check if it's the first trading date of that year
2. ✅ Check if we have a previous year
3. ✅ Get the last-paid dividend of prev. year := `last_paid_div`
4. ✅ Get count of dividends payed of last year per stock := `num_div_paid`
5. ✅ last_paid_div * num_div_paid := `projected_div_amount`
6. ✅Apply weights (how? => all in on highest value stock)

#### Strategy implementation - Dogs of the Stocks

In [69]:
class DogsOfTheStocks(Strategy):
    def __init__(self, top_n_stocks: int = 10, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.top_n_stocks = top_n_stocks
    
    def weight(self, market_state: pd.DataFrame) -> Dict[str, float]:
        # make default weight map
        weights = {}
        stocks = set(market_state.symbol.unique())
        for stock in stocks:
            weights[stock] = 0.
        
        current_date = market_state.date.max()
        if current_date.year == market_state.date.min().year:
            # there is no previous year
            return weights
        
        if current_date != market_state.loc[market_state.date.dt.year == current_date.year].date.min():
            # it's not the first trading day of the current year
            return weights
        
        prev_year = current_date.year - 1
        last_paid_divs = {}
        # all dividends paid last year
        dividends_paid_last_year_mask = (market_state.dividend_amount != 0.) & (market_state.date.dt.year == prev_year)
        divs = market_state.loc[dividends_paid_last_year_mask].sort_values(by="date")
        div_yields = []
        for stock in stocks:
            # for every stock:
            # > get last paid dividend of prev. year
            # > get amount of times dividends were paid last year
            last_paid_divs[stock] = {}
            try:
                # if there were dividends paid last year, grab the last one
                last_paid_divs[stock]["amount"] = divs.loc[divs.symbol == stock].dividend_amount.values[-1]
            except IndexError:
                # there were no dividends paid last year
                last_paid_divs[stock]["amount"] = 0
            last_paid_divs[stock]["times_paid"] = len(divs.loc[divs.symbol == stock])

        for stock, dividend_data in last_paid_divs.items():
            # for every stock: multiply last paid dividend by number of times div
            # were paid last year,
            # and divide by the current stock price
            current_price_mask = (market_state.date == current_date) &\
                (market_state.symbol == stock)
            current_price = market_state.loc[current_price_mask].close.values[0]
            current_stock = (stock, dividend_data["amount"] * dividend_data["times_paid"] / current_price)
            div_yields.append(current_stock)

            # pick stock with highest relative dividend value,
            # and  assign weight of 1
            # TODO(jonas): weight relative to %share of total
        div_yields.sort(key=lambda t: t[1], reverse=True)
        div_yields = div_yields[:self.top_n_stocks]
        total_yield_sum = sum([t[1] for t in div_yields])
        for symbol, div_yield in div_yields:
            weights[symbol] = div_yield / total_yield_sum
        
        return weights

### Agent

In [59]:
class Agent:
    def __init__(self, starting_capital: float, market: Market, strategy: Strategy, name: str):
        # TODO(jonas): implement agent getting new cash every [x interval]
        self.cash = starting_capital
        self.market = market
        self.state = None
        self.strategy = strategy
        self.name = name
        self.portfolio = {}
        self.trading_history = []
    
    def run_simulation(self):
        for state in market:
            self.state = state
            # agent gets dividends paid out
            self.cash += self.market.pay_dividends(stock_portfolio=self.portfolio)
            weights = self.strategy.weight(market_state=self.state)
            for symbol, weight in weights.items():
                if weight != 0:
#                     assert sum(weights.values()) == 1, "Not 1!"
                    print(weights)
                    break
                    
    def buy(self, stock_symbol: str, stock_price: float, amount: int, date):
        total_price = amount * stock_price
        assert self.cash - total_price >= 0, "Can't spend more than you have!"
        self.cash -= total_price
        self.portfolio[stock_symbol] = self.portfolio.get(stock_symbol, 0) + amount
        transaction = Transaction(
            symbol=stock_symbol,
            amount=amount,
            date=date,
            stock_price=stock_price,
            total_value=total_price,
        )
        self.trading_history.append(transaction)

### Definition of a trading transaction for the agent

In [13]:
Transaction = namedtuple(
    "Transaction", 
    field_names=["symbol", "amount", "date", "stock_price", "total_value"]
)

### Testing

In [70]:
market = Market(prices=df.loc[df.date.dt.year.isin([2017, 2018, 2019, 2020])], name="test")
strategy = DogsOfTheStocks(name="DogsOfTheStocks")
agent = Agent(
    starting_capital=10_000,
    market=market,
    strategy=strategy,
    name="test"
)

agent.run_simulation()

C:\Users\Jonas\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


{'MA': 0.0, 'YUM': 0.0, 'SBUX': 0.0830151364995818, 'ABC': 0.0, 'ANTM': 0.0, 'ALB': 0.0, 'MCD': 0.09298394988330076, 'AOS': 0.0, 'GD': 0.0, 'TPR': 0.11909315532237176, 'IPGP': 0.0, 'CB': 0.07956793733179522, 'ESS': 0.11624297531134166, 'PFE': 0.14004134113343283, 'GWW': 0.086599744956129, 'WAT': 0.0, 'LOW': 0.07136384158309933, 'HSY': 0.09331342668227365, 'CPB': 0.11777849129667402, 'IFF': 0.0}
{'MA': 0.0, 'YUM': 0.0, 'SBUX': 0.07990537963517216, 'ABC': 0.0, 'ANTM': 0.0, 'ALB': 0.0, 'MCD': 0.09406257117263932, 'AOS': 0.0, 'GD': 0.08442755869375687, 'TPR': 0.10517507859448465, 'IPGP': 0.0, 'CB': 0.0818743995172445, 'ESS': 0.11223701661253818, 'PFE': 0.11223087772419095, 'GWW': 0.0, 'WAT': 0.0, 'LOW': 0.0, 'HSY': 0.09775778539382442, 'CPB': 0.1548899485385234, 'IFF': 0.07743938411762559}
{'MA': 0.0, 'YUM': 0.0, 'SBUX': 0.0, 'ABC': 0.0, 'ANTM': 0.0, 'ALB': 0.07360522117016303, 'MCD': 0.09053482128526907, 'AOS': 0.07306404561538374, 'GD': 0.08313425545563664, 'TPR': 0.18225738680967984, 'I

In [127]:
t = Transaction(symbol="AAPL", amount=10, date="2020-01-01", stock_price=3000.00, total_value=30_000.00)

In [128]:
t.amount

10

# To-Do list
1. ✅ Make agent more intelligent (weighting of stocks)
2. ⬜ implement agent getting new cash every x interval
3. ✅ agent gets dividends
4. ✅ dividend amount adapts to amount of stocks agent has
5. ⬜ agent buys stuff